<a href="https://colab.research.google.com/github/hieulv416c/Url_content_detect/blob/main/Doc2vec%2BMLnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install --upgrade gensim
import pandas as pd
import numpy as np
import time
import re
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression





In [ ]:
# Load dữ liệu
df = pd.read_csv("/content/content_cleaned.csv")
df["Content"] = df["Content"].astype(str)

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    df["Content"].fillna(""), df["result"].fillna(0), test_size=0.2, random_state=42
)

# Gán nhãn cho dữ liệu để huấn luyện Doc2Vec
train_tagged = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(X_train)]
test_tagged = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(X_test)]

# Khởi tạo mô hình Doc2Vec trước khi build_vocab
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, epochs=20)

# Đo thời gian huấn luyện Doc2Vec
start_doc2vec_time = time.time()

print("Huấn luyện Doc2Vec...")
doc2vec_model.build_vocab(train_tagged)
doc2vec_model.train(train_tagged, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

end_doc2vec_time = time.time()
doc2vec_training_time = end_doc2vec_time - start_doc2vec_time

print(f"⏳ Thời gian huấn luyện Doc2Vec: {doc2vec_training_time:.2f} giây")




Huấn luyện Doc2Vec...
⏳ Thời gian huấn luyện Doc2Vec: 1546.95 giây


In [ ]:
# Chuyển văn bản thành vector
def vectorize_doc2vec(model, tagged_docs):
    return np.array([model.infer_vector(doc.words) for doc in tagged_docs])

X_train_vec = vectorize_doc2vec(doc2vec_model, train_tagged)
X_test_vec = vectorize_doc2vec(doc2vec_model, test_tagged)

# Danh sách các mô hình cần thử nghiệm (trừ Random Forest, SVM, Logistic Regression)
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel="linear", probability=True),
    "Logistic Regression": LogisticRegression(max_iter=300)
}
# Hàm huấn luyện và đánh giá mô hình
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    start_train_time = time.time()
    model.fit(X_train, y_train)
    end_train_time = time.time()

    start_eval_time = time.time()
    y_pred = model.predict(X_test)
    end_eval_time = time.time()

    acc = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")

    print(f"Accuracy: {acc:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1:.4f}")
    print(f"Thời gian huấn luyện: {end_train_time - start_train_time:.2f} giây")
    print(f"Thời gian đánh giá: {end_eval_time - start_eval_time:.2f} giây\n")

# Chạy thử nghiệm với các mô hình
for name, model in models.items():
    print(f"🚀 Đang huấn luyện mô hình: {name}")
    train_and_evaluate(model, X_train_vec, y_train, X_test_vec, y_test)
